<h1> <center> Amazon Fake Reviews </center> </h1>

In [1]:
#!pip install langdetect
#!pip install language_check

In [2]:
#General purpose
import os
#Data handling
import pandas as pd
import numpy as np
import datetime
#Data Visualization
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline 
#Modelling 
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from langdetect import detect #to filter non-english reviews.
import language_check #to detect errors
import sklearn
from sklearn.feature_extraction.text import CountVectorizer

In [3]:
## nltk downloads
# nltk.download('punkt') # At first you have to download these nltk packages.
# nltk.download('stopwords')
# nltk.download('wordnet')

In [4]:
#Load dataset
df=pd.read_csv('../data/exports/test_appliances.csv')
df.head()

,asin,reviewDate,reviewerID,reviewerName,reviewTitle,productGradeByReview,reviewText,reviewUpvotes,reviewHasImage,reviewIsVerified,...,productTitle,productDescription,productFeatures,productHasImage,productCategory,productBrand,productRank,productYear,productMonth,productDay
0,B00009W3I4,2013-08-22,A34A1UP40713F8,James. Backus,Great product,5,I like this as a vent as well as something tha...,0,0,1,...,HEARTLAND 21000 Energy Saving Dryer Vent Closure,Replace your old vent with the Heartland Natur...,EASY-TO-INSTALL Dryer vent closure with hassl...,1.0,Tools & Home Improvement,HEARTLAND,5633.0,2003.0,5.0,14.0
1,B00009W3PA,2016-02-08,A1AHW6I678O6F2,kevin.,Five Stars,5,good item,0,0,1,...,Certified Appliance Accessories 3-Wire Closed-...,Some people are surprised to learn that a new ...,250 volt,1.0,Tools & Home Improvement,Certified Appliance Accessories,135619.0,2006.0,5.0,26.0
2,B00009W3PA,2015-08-05,A8R48NKTGCJDQ,CDBrannom,Five Stars,5,Fit my new LG dryer perfectly.,0,0,1,...,Certified Appliance Accessories 3-Wire Closed-...,Some people are surprised to learn that a new ...,250 volt,1.0,Tools & Home Improvement,Certified Appliance Accessories,135619.0,2006.0,5.0,26.0
3,B00009W3PA,2015-04-24,AR3OHHHW01A8E,Calvin E Reames,Perfect size,5,Good value for electric dryers,0,0,1,...,Certified Appliance Accessories 3-Wire Closed-...,Some people are surprised to learn that a new ...,250 volt,1.0,Tools & Home Improvement,Certified Appliance Accessories,135619.0,2006.0,5.0,26.0
4,B00009W3PA,2015-03-21,A2CIEGHZ7L1WWR,albert j. kong,Five Stars,5,Price and delivery was excellent.,0,0,1,...,Certified Appliance Accessories 3-Wire Closed-...,Some people are surprised to learn that a new ...,250 volt,1.0,Tools & Home Improvement,Certified Appliance Accessories,135619.0,2006.0,5.0,26.0


In [5]:
df.columns

Index(['asin', 'reviewDate', 'reviewerID', 'reviewerName', 'reviewTitle',
       'productGradeByReview', 'reviewText', 'reviewUpvotes', 'reviewHasImage',
       'reviewIsVerified', 'reviewYear', 'reviewMonth', 'reviewDay',
       'productDate', 'productTitle', 'productDescription', 'productFeatures',
       'productHasImage', 'productCategory', 'productBrand', 'productRank',
       'productYear', 'productMonth', 'productDay'],
      dtype='object')

# Useful functions

In [6]:
#Print review
def print_review(df,i):
    print('-------------------')
    print('Review #{}'.format(i))
    print('Title: {},\nReview: {},\nReviewer ID: {},\nVerified: {}'
          .format(df['reviewTitle'][i], df['reviewText'][i], df['reviewerID'][i], df['reviewIsVerified'][i]))
    

In [7]:
#Print product
def print_product(df,i):
    print('-------------------')
    print('Product: {}'.format(df['asin'][i]))
    print('Name: {},\nCategory: {},\nRank: {},\nBrand: {}\nDescription: {}'
         .format(df['productTitle'][i],df['productCategory'][i],df['productRank'][i],df['productBrand'][i],df['productDescription'][i]))
#ToDo
#Modify to be able to get and print a product by its ID

In [8]:
#Cosine Similarity
def cosine_sim_vectors(vec1,vec2):
    vec1=vec1.reshape(1,-1)
    vec2=vec2.reshape(1,-1)
    
    return cosine_similarity(vec1,vec2)[0][0]

In [40]:
#Print error
def print_error(matches,i):
    error=matches[i]
    print('-------------------')
    print('Rule ID: ',error.ruleId)
    print('Category: ',error.category)
    print('Error and context: ',error.context)
    print(error.msg)

In [44]:
#Manage errors
def manage_errors(text,matches=None):
    if matches is None:
        matches=language_check.LanguageTool('en-US').check(text)
    mistakes=[]
    corrections=[]
    error_start=[]
    error_end=[]
    
    for error in matches:
        if len(error.replacements)>0:
            error_start.append(error.offset)
            error_end.append(error.offset+error.errorlength)
            mistakes.append(text[error_start[-1]:error_end[-1]])
            corrections.append(error.replacements[0])
            
    #return list(zip(mistakes,corrections,error_start,error_end))
    return mistakes,corrections,error_start,error_end

In [58]:
#Correct errors
def correct_errors(text,matches=None, mistakes=None,corrections=None,start=None, end=None):
    
    if end is None:
        mistakes,corrections,start,end=manage_errors(text,matches)
        
    corrected_text=list(text)
    
    for i in range(len(start)):
        for j in range(len(text)):
            if (j>start[i] and j<end[i]):
                corrected_text[j]=""
            
        corrected_text[start[i]]=corrections[i]
        
    return "".join(corrected_text)
        

# Data Analysis

In [12]:
df.dtypes

asin                     object
reviewDate               object
reviewerID               object
reviewerName             object
reviewTitle              object
productGradeByReview      int64
reviewText               object
reviewUpvotes             int64
reviewHasImage            int64
reviewIsVerified          int64
reviewYear                int64
reviewMonth               int64
reviewDay                 int64
productDate              object
productTitle             object
productDescription       object
productFeatures          object
productHasImage         float64
productCategory          object
productBrand             object
productRank             float64
productYear             float64
productMonth            float64
productDay              float64
dtype: object

* reviewDate and productDate should be changed to datetime
* The categorical variables (productGradeByReview, reviewUpvotes*, reviewHasImage, reviewIsVerified could be change temporarily to float to use the Pandas Describe function (cf cell below)

In [13]:
df.describe(include='all')

,asin,reviewDate,reviewerID,reviewerName,reviewTitle,productGradeByReview,reviewText,reviewUpvotes,reviewHasImage,reviewIsVerified,...,productTitle,productDescription,productFeatures,productHasImage,productCategory,productBrand,productRank,productYear,productMonth,productDay
count,2277,2277,2277,2277,2277,2277.000000,2277,2277.000000,2277.000000,2277.000000,...,2275,190,2245,2275.000000,2275,2275,2.274000e+03,2275.000000,2275.000000,2275.000000
unique,48,99,47,47,96,NaN,144,NaN,NaN,NaN,...,46,41,40,NaN,5,26,NaN,NaN,NaN,NaN
top,B0006GVNOA,2016-12-06,A1WD61B0C3KQZB,J. Foust,A great device that has my dryer working well ...,NaN,Great product but they need to include more ro...,NaN,NaN,NaN,...,Gardus RLE202 LintEater Rotary Dryer Vent Clea...,Constructed of 4-ply aluminum foil laminate. F...,Most valuable tool you can own to ensure the s...,NaN,Tools &amp; Home Improvement,LintEater,NaN,NaN,NaN,NaN
freq,2040,412,206,206,206,NaN,206,NaN,NaN,NaN,...,2040,12,2064,NaN,2040,2064,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,4.499780,NaN,221.378568,0.363636,0.987264,...,NaN,NaN,NaN,0.952527,NaN,NaN,6.880796e+03,2001.767473,4.291868,7.040879
std,NaN,NaN,NaN,NaN,NaN,0.837723,NaN,542.987888,0.481151,0.112158,...,NaN,NaN,NaN,0.212694,NaN,NaN,4.348922e+04,72.809410,1.185184,4.175630
min,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,0.000000,0.000000,0.000000,...,NaN,NaN,NaN,0.000000,NaN,NaN,1.000000e+00,-1.000000,-1.000000,-1.000000
25%,NaN,NaN,NaN,NaN,NaN,4.000000,NaN,5.000000,0.000000,1.000000,...,NaN,NaN,NaN,1.000000,NaN,NaN,1.976000e+03,2004.000000,4.000000,6.000000
50%,NaN,NaN,NaN,NaN,NaN,5.000000,NaN,8.000000,0.000000,1.000000,...,NaN,NaN,NaN,1.000000,NaN,NaN,1.976000e+03,2004.000000,4.000000,6.000000
75%,NaN,NaN,NaN,NaN,NaN,5.000000,NaN,165.000000,1.000000,1.000000,...,NaN,NaN,NaN,1.000000,NaN,NaN,1.976000e+03,2004.000000,4.000000,6.000000


* The dataset contains 2277 reviews for 48 different products and from 47 different reviewers
* The categorical variables are not analyzed properly with the describe method => A solution would be to convert them to float
* 75% of the reviews assign a grade of 5 to the products
* We know from before that most of the reviews are verified with a ratio around 1000/1.


In [14]:
#Confirming that we have reviews for only 48 products
df['asin'].value_counts().shape

(48,)

In [15]:
df['asin'].value_counts().head()

B0006GVNOA    2040
B0014CN8Y8      24
B00JGTUQJ6      13
B0015UGPWQ      12
B0056I99WG       8
Name: asin, dtype: int64

In [16]:
#Let's examine this product
temp=df[df['asin']=='B0006GVNOA'].head(1).reset_index()
print_product(temp,0)

-------------------
Product: B0006GVNOA
Name: Gardus RLE202 LintEater Rotary Dryer Vent Cleaning System,
Category: Tools &amp; Home Improvement,
Rank: 1976.0,
Brand: LintEater
Description: nan


# Data Processing

## Filtering on English reviews

In [17]:
df['reviewLang']=df['reviewText'].apply(lambda x : detect(x)) 
df.head()

,asin,reviewDate,reviewerID,reviewerName,reviewTitle,productGradeByReview,reviewText,reviewUpvotes,reviewHasImage,reviewIsVerified,...,productDescription,productFeatures,productHasImage,productCategory,productBrand,productRank,productYear,productMonth,productDay,reviewLang
0,B00009W3I4,2013-08-22,A34A1UP40713F8,James. Backus,Great product,5,I like this as a vent as well as something tha...,0,0,1,...,Replace your old vent with the Heartland Natur...,EASY-TO-INSTALL Dryer vent closure with hassl...,1.0,Tools & Home Improvement,HEARTLAND,5633.0,2003.0,5.0,14.0,en
1,B00009W3PA,2016-02-08,A1AHW6I678O6F2,kevin.,Five Stars,5,good item,0,0,1,...,Some people are surprised to learn that a new ...,250 volt,1.0,Tools & Home Improvement,Certified Appliance Accessories,135619.0,2006.0,5.0,26.0,en
2,B00009W3PA,2015-08-05,A8R48NKTGCJDQ,CDBrannom,Five Stars,5,Fit my new LG dryer perfectly.,0,0,1,...,Some people are surprised to learn that a new ...,250 volt,1.0,Tools & Home Improvement,Certified Appliance Accessories,135619.0,2006.0,5.0,26.0,en
3,B00009W3PA,2015-04-24,AR3OHHHW01A8E,Calvin E Reames,Perfect size,5,Good value for electric dryers,0,0,1,...,Some people are surprised to learn that a new ...,250 volt,1.0,Tools & Home Improvement,Certified Appliance Accessories,135619.0,2006.0,5.0,26.0,en
4,B00009W3PA,2015-03-21,A2CIEGHZ7L1WWR,albert j. kong,Five Stars,5,Price and delivery was excellent.,0,0,1,...,Some people are surprised to learn that a new ...,250 volt,1.0,Tools & Home Improvement,Certified Appliance Accessories,135619.0,2006.0,5.0,26.0,en


In [18]:
df['reviewLang'].value_counts()

en    2234
so      15
ro       7
hu       5
de       4
cs       2
af       2
sk       1
pl       1
sl       1
id       1
fi       1
ca       1
et       1
sw       1
Name: reviewLang, dtype: int64

In [19]:
features=df[df['reviewLang']=='en']
features.head()

,asin,reviewDate,reviewerID,reviewerName,reviewTitle,productGradeByReview,reviewText,reviewUpvotes,reviewHasImage,reviewIsVerified,...,productDescription,productFeatures,productHasImage,productCategory,productBrand,productRank,productYear,productMonth,productDay,reviewLang
0,B00009W3I4,2013-08-22,A34A1UP40713F8,James. Backus,Great product,5,I like this as a vent as well as something tha...,0,0,1,...,Replace your old vent with the Heartland Natur...,EASY-TO-INSTALL Dryer vent closure with hassl...,1.0,Tools & Home Improvement,HEARTLAND,5633.0,2003.0,5.0,14.0,en
1,B00009W3PA,2016-02-08,A1AHW6I678O6F2,kevin.,Five Stars,5,good item,0,0,1,...,Some people are surprised to learn that a new ...,250 volt,1.0,Tools & Home Improvement,Certified Appliance Accessories,135619.0,2006.0,5.0,26.0,en
2,B00009W3PA,2015-08-05,A8R48NKTGCJDQ,CDBrannom,Five Stars,5,Fit my new LG dryer perfectly.,0,0,1,...,Some people are surprised to learn that a new ...,250 volt,1.0,Tools & Home Improvement,Certified Appliance Accessories,135619.0,2006.0,5.0,26.0,en
3,B00009W3PA,2015-04-24,AR3OHHHW01A8E,Calvin E Reames,Perfect size,5,Good value for electric dryers,0,0,1,...,Some people are surprised to learn that a new ...,250 volt,1.0,Tools & Home Improvement,Certified Appliance Accessories,135619.0,2006.0,5.0,26.0,en
4,B00009W3PA,2015-03-21,A2CIEGHZ7L1WWR,albert j. kong,Five Stars,5,Price and delivery was excellent.,0,0,1,...,Some people are surprised to learn that a new ...,250 volt,1.0,Tools & Home Improvement,Certified Appliance Accessories,135619.0,2006.0,5.0,26.0,en


In [20]:
features['reviewLang'].value_counts()

en    2234
Name: reviewLang, dtype: int64

## Checking grammar and syntax errors 

In [21]:
checker = language_check.LanguageTool('en-US')

In [22]:
text="LanguageTool offers spell and grammar checking. Just paste your text here and click the ‘Check Text’ button. Click the colored phrases for details on potential errors. or use this text too see an few of of the problems that LanguageTool can detecd. What do you thinks of grammar checkers? Please not that they are not perfect. Style issues get a blue marker: It’s 5 P.M. in the afternoon. The weather was nice on Thursday, 27 June 2017"

In [24]:
#matches=checker.check(text)
matches

[Match({'fromy': 0, 'fromx': 168, 'toy': 0, 'tox': 170, 'ruleId': 'UPPERCASE_SENTENCE_START', 'msg': 'This sentence does not start with an uppercase letter', 'replacements': ['Or'], 'context': '...hrases for details on potential errors. or use this text too see an few of of the ...', 'contextoffset': 43, 'offset': 168, 'errorlength': 2, 'category': 'Capitalization', 'locqualityissuetype': 'typographical'}),
 Match({'fromy': 0, 'fromx': 185, 'toy': 0, 'tox': 192, 'ruleId': 'TOO_TO', 'subId': '1', 'msg': "Did you mean 'to see'?", 'replacements': ['to see'], 'context': '...s on potential errors. or use this text too see an few of of the problems that Language...', 'contextoffset': 43, 'offset': 185, 'errorlength': 7, 'category': 'Possible Typo', 'locqualityissuetype': 'misspelling'}),
 Match({'fromy': 0, 'fromx': 193, 'toy': 0, 'tox': 195, 'ruleId': 'EN_A_VS_AN', 'msg': "Use 'a' instead of 'an' if the following word doesn't start with a vowel sound, e.g. 'a sentence', 'a university'", 're

In [41]:
print_error(matches,0)

-------------------
Rule ID:  UPPERCASE_SENTENCE_START
Category:  Capitalization
This sentence does not start with an uppercase letter
Error and context:  ...hrases for details on potential errors. or use this text too see an few of of the ...


In [39]:
manage_errors(text,matches)

[('or', 'Or', 168, 170),
 ('too see', 'to see', 185, 192),
 ('an', 'a', 193, 195),
 ('of of', 'of', 200, 205),
 ('detecd', 'detect', 241, 247),
 ('thinks', 'think', 261, 267),
 ('not', 'note', 296, 299),
 ('P.M. in the afternoon', 'P.M.', 366, 387)]

In [59]:
correct_errors(text,matches)

'LanguageTool offers spell and grammar checking. Just paste your text here and click the ‘Check Text’ button. Click the colored phrases for details on potential errors. Or use this text to see a few of the problems that LanguageTool can detect. What do you think of grammar checkers? Please note that they are not perfect. Style issues get a blue marker: It’s 5 P.M.. The weather was nice on Thursday, 27 June 2017'